### Задание 1. Обязательная часть

Из страницу с новостями на habr.com/ru/all/ нужны статьи, в которых есть ключевые слова KEYWORDS. 

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

*В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
import time
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [65]:
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['telegram', 'Go-вариант']

In [66]:
news = requests.get(URL)
soup = BeautifulSoup(news.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [67]:
for post in posts:
    
    post_lower = post.find('div', class_='post__body').text.lower()
    title_element = post.find('a', class_='post__title_link').text
    link = post.find('a', class_='post__title_link').attrs.get('href')
    
    for key in KEYWORDS:
        
        if key.lower() in post_lower or key.lower() in title_element:

            date_element = post.find('span', class_='post__time').text
            print(date_element, title_element, link)

сегодня в 16:28 Telegram для маркетолога: зачем использовать чаты для продвижения бизнеса и как это делать https://habr.com/ru/post/515810/
сегодня в 16:16 Разработка инструмента командной строки: сравнение Go и Rust https://habr.com/ru/company/ruvds/blog/515674/


In [68]:
# дополнительное задание
def get_all_links(url):
    all_refs = []
    
    res = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')

    articles = soup.find_all('article', class_='post')
    all_refs = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), articles))

    return all_refs

all_links = get_all_links(URL)
all_links[:3]

['https://habr.com/ru/company/pt/blog/515842/',
 'https://habr.com/ru/company/etmc_exponenta/blog/515850/',
 'https://habr.com/ru/company/epam_systems/blog/515600/']

In [69]:
kom_news = pd.DataFrame()
for link in all_links:

    soup_new = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)

    soup_new_low = soup_new.find('div', class_='post__body').text.lower()
    title = soup_new.find('title').text

    for key in KEYWORDS:
        
        if key.lower() in soup_new_low or key.lower() in title:
        
            date = pd.to_datetime(soup_new.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
            row = {'date': date, 'title': title, 'link': link}
            kom_news = pd.concat([kom_news, pd.DataFrame([row])]) 
kom_news

,date,title,link
0,2020-08-19,Telegram для маркетолога: зачем использовать ч...,https://habr.com/ru/post/515810/
0,2020-08-19,Разработка инструмента командной строки: сравн...,https://habr.com/ru/company/ruvds/blog/515674/


### Задание 2. Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 

*Датафрейм должен иметь столбцы: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [142]:
import json

In [193]:
EMAIL = ['ada@severtm.ru', 'zobr2005@mail.ru', 'd.anchunov@gmail.com']

In [210]:
def checking_your_email(e_mail):
    URL = 'https://digibody.avast.com/v1/web/leaks'
    data = {
        'email': e_mail
    }
    req = requests.post(URL, json=data)
    your_passwords_have_leaked = pd.DataFrame()
    
    for q in req.json()['value']:
        
        date = time.ctime(q['leak_date']/1000)
        title = q['leak_info']['title']
        info = q['leak_info']['description']

        row = {'email': e_mail, 'date': date, 'title': title, 'info':info}
        your_passwords_have_leaked = your_passwords_have_leaked.append([row])

    return your_passwords_have_leaked

In [209]:
test_result = pd.DataFrame()
for email in EMAIL:
    test_result = test_result.append(checking_your_email(email))
    
test_result.head()

,email,date,title,info
0,zobr2005@mail.ru,Wed Feb 6 03:00:00 2019,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
0,zobr2005@mail.ru,Fri Oct 21 03:00:00 2016,Adobe Systems,"In October of 2013, criminals penetrated Adobe..."
0,zobr2005@mail.ru,Tue Jan 29 03:00:00 2019,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
0,zobr2005@mail.ru,Fri Dec 13 03:00:00 2019,Sensitive Source,This source has been marked as sensitive due t...
0,d.anchunov@gmail.com,Wed Mar 20 03:00:00 2019,Bookmate,"In July 2018, Bookmate was allegedly breached...."
